# Practical 6: Cell-Cell Communication

Author: Francesca Drummer

In this notebook we will cover different methods to revocer cell-cell communication (CCC) in spatial transcriptomics. 

1. non-spatial CCC testing with spatial DE genes or post-processing filter (e.i. spatial distance) using CellPhoneDB
2. MISTy

To reduce the environment dependencies we will use the LIANA+ implementation of the methods. 
Please notice that the original tools might offer more functionalities. 
For that reason we will always link to the original publication and GitHub repository.

In [50]:
import squidpy as sq
import scanpy as sc

from pathlib import Path
import numpy as np

from scipy.sparse import issparse, csr_matrix

from liana.method import singlecellsignalr, connectome, cellphonedb, natmi, logfc, cellchat, geometric_mean

## 0. Download data

We will use the **Xenium AD dataset** from the previous notebooks here.

As a reminder the dataset consists of 6 coronal mouse brain slices from 2 different conditions (wildtype - ctrl vs TgCRND8 - AD) across 3 timepoints. In this practical, we additionally have information about cell types available in  `adata.obs['cell_types']`. Please note that these annotation are not perfect. For example, there are quite some cells that could not be assigned to a cell type (NaN or "unkown"). These annotations have been made with on leiden clustering and marker genes reported in [this](https://pages.10xgenomics.com/rs/446-PBO-704/images/10x_LIT000210_App-Note_Xenium-In-Situ_Letter_Digital.pdf) document. 

In this practical we aim to understand the differences of the mouse brain between the two conditions and across the timepoints using niches and spatial domains.

In [66]:
PATH = "/data/spatial_workshop/day3/practical_4"

In [ ]:
# load adata
adata = sc.read_h5ad(Path(PATH, 'xenium_mouse_ad_annotated_rotated_domain.h5ad'))
adata

In [ ]:
print(adata.X[:2,:2])

In [69]:
adata.layers['counts'] = adata.X

In [70]:
# Normalization to the median
sc.pp.normalize_total(adata)

# Freeman-Tukey square root transform
assert issparse(adata.X)
sqrt_X = adata.X.sqrt()
# Create a new sparse matrix for X + 1
X_plus_1 = adata.X + csr_matrix(np.ones(adata.X.shape))
# Calculate the square root of (X + 1)
sqrt_X_plus_1 = X_plus_1.sqrt()
adata.layers['median_ft'] = sqrt_X + sqrt_X_plus_1

## 0. Introduction to LIANA+

[LIANA+](https://liana-py.readthedocs.io/en/latest/notebooks/basic_usage.html) is a toolbox in Python for various dissociated, multimodal and spatially informed cell-cell communication tools [Dimitrov et al., 2024]. 

First we install the package and observe which methods are implemented in LIANA+. 
Each method relies on different assumptions and returns a different ligand-receptor score. 
Usually, one score for the strength of the interaction (`magnitude`) and a score reflecting the `specifivity` of a interaction to a pair of cell identities. 

In [ ]:
import liana as li

li.mt.show_methods()

Most CCC tools identify LR interaction. For this they rely on a extracting LR pairs from databases. There are diverse databases but LIANA+ has a consensus database that uses LR that are overlapping across databases. 

First, we need to ensure that there are LR-pairs present in the data to be detected for communication.

In [ ]:
print(li.resource.show_resources())
resource_name = "mouseconsensus"  # Replace with the desired resource name if needed
lr_pairs = li.resource.select_resource(resource_name)
lr_pairs

In [73]:
def lr_pairs_in_adata(adata):
    genes_in_dataset = set(adata.var_names)  # Replace `adata.var_names` with your dataset's gene names if different
    
    # Filter the ligand-receptor pairs for those present in your dataset
    filtered_lr_pairs = lr_pairs[
        lr_pairs['ligand'].isin(genes_in_dataset) & lr_pairs['receptor'].isin(genes_in_dataset)
    ]
    
    return filtered_lr_pairs
    

In [74]:
filtered_lr_pairs = lr_pairs_in_adata(adata)

<span style="color:red; font-weight:bold">Task 1: How many ligand-receptor pairs are in the data?</span>

In the following chapter, we will work with the CellPhoneDB method from LIANA+.

## 1. CellPhoneDB: non-spatial CCC

First, we will run CellPhoneDB as if we did not have any spatial information. 

In [ ]:
sub_adata = adata[(adata.obs['time'] == '5_7') & (adata.obs['condition'] == 'TgCRND8')]
sub_adata

In [ ]:
cellphonedb(sub_adata,
            groupby='cell_types',
            # NOTE by default the resource uses HUMAN gene symbols
            resource_name='mouseconsensus',
            expr_prop=0.1,
            verbose=True, 
            use_raw = False,
            layer = 'counts',
            key_added='cpdb_res')

In [ ]:
sub_adata.uns['cpdb_res'].head()

<div style="border: 1px solid #0000ff; padding: 10px; border-radius: 5px;">
<span style="color: #0000ff; font-size: 20px;"><b>Interpretation</b></span> <span style="font-size: 20px;">Liana+ scores</span>  

<span></span>
<ul>
    <li>source and target columns represent the source/sender and target/receiver cell identity for each interaction, respectively</li>
    <li>*_props: represents the proportion of cells that express the entity.</li>
    <li>*_means: entity expression mean per cell type.</li>
    <li>lr_means: mean ligand-receptor expression, as a measure of ligand-receptor interaction magnitude</li>
</ul>

<span style="color:red; font-weight:bold">Task 2: Plot the top 3 interacting complexes</span>

In [ ]:
## TODO
sq.pl.spatial_scatter(sub_adata, 
                      color=[],
                      layer = 'median_ft',
                     shape=None)

In [ ]:
my_plot = li.pl.tileplot(adata = sub_adata,
                         fill='means',
                         label='props',
                         label_fun=lambda x: f'{x:.2f}',
                         top_n=30,
                         orderby='cellphone_pvals',
                         orderby_ascending=True,
                         source_labels=['Astrocytes', 'Excitatory neurons', 'Inhibitory neurons', 'Microglia', 'OPC', 'Oligodendrocytes'],
                         target_labels=['Astrocytes', 'Excitatory neurons', 'Inhibitory neurons', 'Microglia', 'OPC', 'Oligodendrocytes'],
                         uns_key='cpdb_res', # NOTE: default is 'liana_res'
                         source_title='Ligand',
                         target_title='Receptor',
                         figure_size=(8, 7)
                         )
my_plot

<span style="color:red; font-weight:bold">Question: What can we observe if we do not consider spatial information? Why could this be problematic?</span>

To overcome this issue we will cover two possible appraoches to integrate spatial information into non-spatially aware CCC tools, like `CellPhoneDB`.

1. Restrict the input to spatially variable genes. 
2. Post-processing of interactions using spatial proximity, e.i. niche information. 

### Spatially-variable gene selection

We use Moran's I score as a measure of spatial autocorrelation to identify spatially variable genes. 

For more information see: [Chapter 29: Spatially variable genes](https://www.sc-best-practices.org/spatial/spatially_variable_genes.html) from single-cell best practices.

1. Calculate a spatial graph (`sq.gr.spatial_neighbors`)
2. Calculate autocorrelation with [Morans I score](https://squidpy.readthedocs.io/en/stable/notebooks/examples/graph/compute_moran.html) (`sq.gr.spatial_autocorr`)

In [ ]:
print(sub_adata.X[:5,:5])

In [85]:
sq.gr.spatial_neighbors(sub_adata, n_neighs=30, coord_type="generic", key_added = 'neighs_based_spatial')


In [ ]:
sq.gr.interaction_matrix(sub_adata, cluster_key="cell_types", connectivity_key = 'neighs_based_spatial', normalized=True)
sq.pl.interaction_matrix(sub_adata, cluster_key="cell_types")

In [ ]:
sq.gr.spatial_autocorr(sub_adata, connectivity_key = "neighs_based_spatial_connectivities", mode="moran", n_perms=50, genes=sub_adata.var_names)

Show and plot the top genes according to Moran's I score autocorrelation.

In [ ]:
sub_adata.uns["moranI"].head()

<div style="border: 1px solid #0000ff; padding: 10px; border-radius: 5px;">
<span style="color: #0000ff; font-size: 20px;"><b>Moran's I score</b></span> <span style="font-size: 20px;"></span>  

<span></span>
<ul>
    <li>I so the Moran’s I,</li>
    <li>pval_norm a p-value under normality assumption.</li>
    <li>var_norm the variance of the Moran’s I under normality assumption.</li>
    <li>{p_val}_{corr_method} the corrected p-values.</li>
</ul>

<span style="color:red; font-weight:bold">Task 3: Plot the 3 genes with the highest I score.</span>

In [ ]:
# TODO
sq.pl.spatial_scatter(sub_adata, 
                      color=[],
                     shape=None)

<span style="color:red; font-weight:bold">Task 4: Subset the data to include only genes that have a Morans I score higher than 0,2 and check that there are still relevant ligand-receptor pairs in the subdata.</span>

In [ ]:
# TODO
sub_adata_svg = sub_adata[:, sub_adata.uns["moranI"]['I'] > ???]
sub_adata_svg

In [ ]:
# TODO


#### CellPhoneDB

In [ ]:
cellphonedb(sub_adata_svg,
            groupby='cell_types',
            # NOTE by default the resource uses HUMAN gene symbols
            resource_name='mouseconsensus',
            expr_prop=0.1,
            verbose=True, 
            use_raw = False,
            layer = 'counts',
            key_added='cpdb_res')

In [ ]:
sub_adata_svg.uns['cpdb_res'].head()

In [ ]:
my_plot = li.pl.tileplot(adata = sub_adata_svg,
                         fill='means',
                         label='props',
                         label_fun=lambda x: f'{x:.2f}',
                         top_n=20,
                         orderby='cellphone_pvals',
                         orderby_ascending=True,
                         source_labels=['Astrocytes', 'Excitatory neurons', 'Inhibitory neurons', 'Microglia', 'OPC', 'Oligodendrocytes'],
                         target_labels=['Astrocytes'],
                         uns_key='cpdb_res', # NOTE: default is 'liana_res'
                         source_title='Ligand',
                         target_title='Receptor',
                         figure_size=(8, 7)
                         )
my_plot

<span style="color:red; font-weight:bold">Question: What could be a potential limitation / problem with this approach?</span>

<span style="color:red; font-weight:bold">Optional Task: Compare the results for the healthy control or different time points. Do the CCC across cell types change?.</span>

<span style="color:red; font-weight:bold">Optional Task: Change the `expr_prop` in the CellPhoneDB function and try out some other tools like CellChat. How does it effect the results?.</span>

### Spatial proximity

An alternative to pre-selecting spatially variable genes is by restricting the cells to be spatially close when they are communicating. For this we will be using the calculated spatial domains from the previous tutorial. 

In [ ]:
sq.pl.spatial_scatter(sub_adata,
                      color = ['cell_types', 'spatial_domain_temp'],
                      shape=None)

<span style="color:red; font-weight:bold">Task 5: Choose a spatial domain cluster that contains a high proportion of the cell types you are interested in to understand the interaction. Tip: also check that the fraction of unkwon cells is low. </span>

In [ ]:
def relative_abundances(adata, group_by, cell_type_key):
    counts = adata.obs.groupby([group_by, cell_type_key]).size().unstack(fill_value=0)
    relative_abundance = counts.div(counts.sum(axis=1), axis=0)
    return relative_abundance

In [ ]:
relative_abundances(sub_adata, group_by='spatial_domain_temp', cell_type_key='cell_types')

In [ ]:
## TODO
domain = 

In [ ]:
sub_adata_domain = sub_adata[sub_adata.obs['spatial_domain_temp'] == domain]
sub_adata_domain

In [ ]:
cellphonedb(sub_adata_domain,
            groupby='cell_types',
            # NOTE by default the resource uses HUMAN gene symbols
            resource_name='mouseconsensus',
            expr_prop=0.1,
            verbose=True, 
            use_raw = False,
            layer = 'counts',
            key_added='cpdb_res')

In [ ]:
sub_adata_domain.uns['cpdb_res'].head()

In [ ]:
my_plot = li.pl.tileplot(adata = sub_adata_domain,
                         fill='means',
                         label='props',
                         label_fun=lambda x: f'{x:.2f}',
                         top_n=20,
                         orderby='cellphone_pvals',
                         orderby_ascending=True,
                         source_labels=['Astrocytes', 'Excitatory neurons', 'Inhibitory neurons', 'Microglia', 'OPC', 'Oligodendrocytes'],
                         target_labels=['Astrocytes', 'Excitatory neurons', 'Inhibitory neurons', 'Microglia', 'OPC', 'Oligodendrocytes'],
                         uns_key='cpdb_res', # NOTE: default is 'liana_res'
                         source_title='Ligand',
                         target_title='Receptor',
                         figure_size=(8, 7)
                         )
my_plot

## 3. MISTY

[MISTy](https://liana-py.readthedocs.io/en/latest/notebooks/misty.html) is a framework that helps understand how different features, such as genes or cell types interact with each other in space. 
For this MISTy uses so called *views*, each describing a different spatial context.

<img src="./figures/MISTy.png" alt="Alt Text" width="500"/>

In [40]:
import scanpy as sc
import decoupler as dc
import plotnine as p9
import liana as li

# Import Helper functions needed to create MISTy objects
from liana.method import MistyData, genericMistyData, lrMistyData

#Import predefined single view models
from liana.method.sp import RandomForestModel, LinearModel, RobustLinearModel

### 3.1 Estimate pathway activities

Before we run MISTy, let’s estimate pathway activities as a way to make the data a bit more interpretable. We will use [decoupler-py](https://academic.oup.com/bioinformaticsadvances/article/2/1/vbac016/6544613) with pathways genesets from [PROGENy](https://www.nature.com/articles/s41467-017-02391-6). See [this](https://decoupler-py.readthedocs.io/en/latest/notebooks/spatial.html) tutorial for details.

In [ ]:
progeny = dc.get_progeny(organism='mouse', top=200)

In [ ]:
dc.run_mlm(
    mat=adata,
    net=progeny,
    source='source',
    target='target',
    weight='weight',
    verbose=True,
    use_raw=False,
)

In [ ]:
# extract progeny activities as an AnnData object
acts_progeny = li.ut.obsm_to_adata(adata, 'mlm_estimate')
acts_progeny

In [ ]:
acts_progeny.var_names

In [ ]:
# Check how the pathway activities look like
for library_id in acts_progeny.obs["batch_key"].unique():
    adata_subset = acts_progeny[acts_progeny.obs["batch_key"] == library_id]
    print(f'Condition: {np.unique(adata_subset.obs["condition"])[0]} and time: {np.unique(adata_subset.obs["time"])[0]}')
    sc.pl.spatial(
        adata_subset, 
        color=['Androgen', 'Estrogen', 'TGFb', 'TNFa', 'VEGF', 'WNT', 'p53'], 
        cmap='RdBu_r', 
        spot_size=10,
    )

### 3.2. Format MISTy object

MISTy objects are in the [MuData](https://github.com/scverse/mudata) (Bredikhin et al., 2021) object with one modality per view. 

The *intra* view is the target variable 

In [ ]:
adata

In [33]:
cell_assignments = adata.obs['cell_types'].astype(str)

In [ ]:
np.unique(cell_assignments)

In [ ]:
import pandas as pd
import anndata as ad

one_hot_data = pd.get_dummies(cell_assignments)

In [36]:
# Step 3: Create AnnData object
adata_ct = ad.AnnData(
    X=one_hot_data.values,  # One-hot encoding matrix
    obs=pd.DataFrame(index=adata.obs_names),  # Cells as `.obs`
    var=pd.DataFrame(index=np.unique(cell_assignments)),  # Cell types as `.var`
)
adata_ct.obsm['spatial'] = adata.obsm['spatial']

In [37]:
# check key cell types
# sc.pl.spatial(adata_ct,
#               color=['OPC'],
#               size=1.3, ncols=2, alpha_img=0,
#               spot_size = 10
#               )

`genericMistyData` constructs a `MuData` object with the intra view and the cell type proportions as the first view. Then it additionally build a 
1. *juxta* view for the spots that are neighbors of each other, and a
2. *para* view for all surrounding spots within a certain radius, or bandwidth.

In [ ]:
misty = genericMistyData(intra=adata_ct, extra=acts_progeny, cutoff=0.05, bandwidth=200, n_neighs=6)
misty

## 3.3 Learn relationship with MISTy

Now that we have constructed the object, we can learn the relationships across views. 

Reationships can be learned by different models (e.i. RandomForrest, LinearModel). The fastest is the Linear model which we will fit here for each target in the intra-view, using the juxta and para views as predictors.

In [ ]:
misty(model=LinearModel, k_cv=10, seed=1337, bypass_intra=True, verbose = True)

By default the results are saved in the `misty` object because `inplace = True`.

The `misty` object does now contain two DataFrames:

- `target_metrics` describes the predictive performance of each view per target
- `interactions` describes the feature importance per view

In [ ]:
misty.uns['target_metrics'].head()

<div style="border: 1px solid #0000ff; padding: 10px; border-radius: 5px;">
<span style="color: #0000ff; font-size: 20px;"><b>target metrics</b></span> <span style="font-size: 20px;"></span>  

<span></span>
<ul>
    <li>intra_R2: prediction performance using intraview</li>
    <li>gain_R2: performance gain when we additionally consider the other views (in addition to intra)</li>
</ul>

In [ ]:
li.pl.target_metrics(misty, stat='gain_R2', return_fig=True)

In [ ]:
li.pl.contributions(misty, return_fig=True)

In [ ]:
(
    li.pl.interactions(misty, view='juxta', return_fig=True, figure_size=(7,5)) +
    p9.scale_fill_gradient2(low = "blue", mid = "white", high = "red", midpoint = 0)
)

<span style="color:red; font-weight:bold">Fit a `RandomForestModel` instead. How does it effect the results and their interpretability?.</span>

# References

[1] Dimitrov D., Schäfer P.S.L, Farr E., Rodriguez Mier P., Lobentanzer S., Badia-i-Mompel P., Dugourd A., Tanevski J., Ramirez Flores R.O. and Saez-Rodriguez J. LIANA+ provides an all-in-one framework for cell–cell communication inference. Nat Cell Biol (2024). https://doi.org/10.1038/s41556-024-01469-w

[2] Li, Z., Wang, T., Liu, P. & Huang, Y. SpatialDM for rapid identification of spatially co-expressed ligand–receptor and revealing cell–cell communication patterns. Nat Commun 14, 3995 (2023).

[3] Bredikhin, D., Kats, I. & Stegle, O. MUON: multimodal omics analysis framework. Genome Biol 23, 42 (2022). https://doi.org/10.1186/s13059-021-02577-8